# Quantum-Enhanced Erosion Simulation

## Core Concept

This notebook implements a **quantum-flavored erosion simulator** that uses:
- **Qiskit Hadamard gates** to make probabilistic erosion decisions at each cell
- **Quantum entanglement** to model spatial correlations in erosion patterns
- **Quantum phase encoding** for rainfall intensity modulation
- **Realistic physics**: stream power law, sediment transport, hillslope diffusion

### Structure
- **Block 1**: Quantum RNG + Terrain Generation
- **Block 2**: Quantum Erosion Physics
- **Block 3**: Demo + Visualization

### Workflow
1. Generate 2D terrain using quantum-seeded RNG
2. Apply rainfall field
3. Route water flow downhill (discharge accumulation)
4. For each cell with rain:
   - Create quantum circuit with Hadamard gate
   - Measure qubit → 0 or 1
   - If 1: apply erosion based on stream power
5. Transport sediment downstream
6. Apply hillslope diffusion
7. Visualize results

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install -q qiskit qiskit-aer numpy scipy matplotlib
print("✓ Packages installed")

In [ ]:
"""
BLOCK 1: QUANTUM RNG + TERRAIN GENERATION

This block contains:
- Quantum random number generation using Qiskit Hadamard gates
- Fractal terrain generation (quantum-seeded)
- Domain warping and ridge sharpening
- Topographic analysis functions
"""
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from scipy import ndimage
import time
import os
import hashlib

# ==============================================================================
# QISKIT IMPORTS
# ==============================================================================

try:
    from qiskit import QuantumCircuit, transpile
    from qiskit_aer import Aer
    HAVE_QISKIT = True
    print("✓ Qiskit imported successfully (qiskit-aer)")
except ImportError:
    try:
        from qiskit import QuantumCircuit, Aer
        HAVE_QISKIT = True
        print("✓ Qiskit imported successfully (legacy)")
    except ImportError:
        HAVE_QISKIT = False
        print("⚠ Qiskit not available, using classical fallback")

# ==============================================================================
# QUANTUM RANDOM NUMBER GENERATION
# ==============================================================================

def qrng_uint32(n, nbits=32):
    """
    Generate n random uint32 values using Qiskit Hadamard gates.
    
    Creates a quantum circuit with nbits qubits, applies Hadamard to all,
    measures, and converts bitstrings to integers.
    
    Args:
        n: number of random values to generate
        nbits: bits per value (default: 32)
    
    Returns:
        Array of n random uint32 values
    """
    if not HAVE_QISKIT:
        return np.random.default_rng().integers(0, 2**32, size=n, dtype=np.uint32)
    
    from qiskit import QuantumCircuit
    try:
        from qiskit_aer import Aer
    except ImportError:
        from qiskit import Aer
    
    # Create circuit with Hadamard superposition
    qc = QuantumCircuit(nbits, nbits)
    qc.h(range(nbits))  # Hadamard on all qubits: |0⟩ → (|0⟩+|1⟩)/√2
    qc.measure(range(nbits), range(nbits))
    
    # Run on simulator
    backend = Aer.get_backend('qasm_simulator')
    seed_sim = int.from_bytes(os.urandom(4), 'little')
    job = backend.run(qc, shots=n, memory=True, seed_simulator=seed_sim)
    result = job.result()
    memory = result.get_memory(qc)
    
    # Convert bitstrings to uint32
    return np.array([np.uint32(int(bits[::-1], 2)) for bits in memory], dtype=np.uint32)


def rng_from_qrng(n_seeds=4, random_seed=None):
    """
    Create NumPy RNG seeded with quantum randomness.
    
    Mixes quantum entropy with OS entropy and current time for
    maximum unpredictability (unless random_seed provided for reproducibility).
    
    Args:
        n_seeds: number of quantum seeds to generate
        random_seed: if provided, use this deterministic seed instead
    
    Returns:
        numpy.random.Generator instance
    """
    if random_seed is not None:
        return np.random.default_rng(int(random_seed))
    
    # Mix quantum + OS entropy
    seeds = qrng_uint32(n_seeds).tobytes()
    mix = seeds + os.urandom(16) + int(time.time_ns()).to_bytes(8, 'little')
    h = hashlib.blake2b(mix, digest_size=8).digest()
    return np.random.default_rng(int.from_bytes(h, 'little'))

# ==============================================================================
# FRACTAL TERRAIN GENERATION
# ==============================================================================

def fractional_surface(N, beta=3.1, rng=None):
    """
    Generate fractal surface with power-law spectrum.
    
    Creates terrain using inverse FFT with power-law amplitude decay.
    Higher beta → smoother large-scale features.
    
    Args:
        N: grid size (NxN)
        beta: power-law exponent (3.0-3.5 for realistic terrain)
        rng: random number generator
    
    Returns:
        Normalized elevation array (0-1)
    """
    rng = rng or np.random.default_rng()
    kx = np.fft.fftfreq(N)
    ky = np.fft.rfftfreq(N)
    K = np.sqrt(kx[:, None]**2 + ky[None, :]**2)
    K[0, 0] = np.inf  # Avoid division by zero
    amp = 1.0 / (K ** (beta/2))
    phase = rng.uniform(0, 2*np.pi, size=(N, ky.size))
    spec = amp * (np.cos(phase) + 1j*np.sin(phase))
    spec[0, 0] = 0.0  # Zero DC component
    z = np.fft.irfftn(spec, s=(N, N))
    lo, hi = np.percentile(z, [2, 98])
    return np.clip((z - lo)/(hi - lo + 1e-12), 0, 1)


def bilinear_sample(img, X, Y):
    """Bilinear interpolation for domain warping."""
    N = img.shape[0]
    x0 = np.floor(X).astype(int) % N
    y0 = np.floor(Y).astype(int) % N
    x1 = (x0+1) % N
    y1 = (y0+1) % N
    dx = X - np.floor(X)
    dy = Y - np.floor(Y)
    return ((1-dx)*(1-dy)*img[x0,y0] + dx*(1-dy)*img[x1,y0] +
            (1-dx)*dy*img[x0,y1] + dx*dy*img[x1,y1])


def domain_warp(z, rng, amp=0.12, beta=3.0):
    """
    Apply domain warping for micro-relief texture.
    
    Distorts coordinates using fractal noise fields.
    Higher amp → more gnarly micro-features.
    
    Args:
        z: input terrain
        rng: random number generator
        amp: warping amplitude (0.10-0.15 typical)
        beta: smoothness of warp field
    
    Returns:
        Warped terrain
    """
    N = z.shape[0]
    u = fractional_surface(N, beta=beta, rng=rng)*2 - 1
    v = fractional_surface(N, beta=beta, rng=rng)*2 - 1
    ii, jj = np.meshgrid(np.arange(N), np.arange(N), indexing='ij')
    Xw = (ii + amp*N*u) % N
    Yw = (jj + amp*N*v) % N
    return bilinear_sample(z, Xw, Yw)


def ridged_mix(z, alpha=0.18):
    """
    Apply ridged fractal mixing for sharp features.
    
    Creates sharp ridges and valleys by folding the noise.
    Higher alpha → craggier terrain.
    
    Args:
        z: input terrain (0-1)
        alpha: ridge strength (0.15-0.20 typical)
    
    Returns:
        Terrain with sharpened features
    """
    ridged = 1.0 - np.abs(2.0*z - 1.0)
    out = (1-alpha)*z + alpha*ridged
    lo, hi = np.percentile(out, [2, 98])
    return np.clip((out - lo)/(hi - lo + 1e-12), 0, 1)


def quantum_seeded_topography(N=256, beta=3.1, warp_amp=0.12, 
                              ridged_alpha=0.18, random_seed=None):
    """
    Generate terrain using quantum RNG.
    
    Complete terrain generation pipeline:
    1. Quantum RNG seeding
    2. Multi-scale fractal generation
    3. Domain warping
    4. Ridge sharpening
    
    Args:
        N: grid size (128-512 typical)
        beta: power-law exponent (3.0-3.5)
        warp_amp: domain warp strength (0.10-0.15)
        ridged_alpha: ridge sharpening (0.15-0.20)
        random_seed: for reproducibility (None = quantum random)
    
    Returns:
        z_norm: normalized elevation (0-1)
        rng: the random generator used
    """
    rng = rng_from_qrng(n_seeds=4, random_seed=random_seed)
    
    # Multi-scale combination
    base_low = fractional_surface(N, beta=beta, rng=rng)
    base_high = fractional_surface(N, beta=beta-0.4, rng=rng)
    z = 0.65*base_low + 0.35*base_high
    
    # Apply domain warp for texture
    z = domain_warp(z, rng=rng, amp=warp_amp, beta=beta)
    
    # Sharpen ridges
    z = ridged_mix(z, alpha=ridged_alpha)
    
    return z, rng

# ==============================================================================
# TOPOGRAPHIC ANALYSIS
# ==============================================================================

def compute_topo_fields(surface_elev, pixel_scale_m):
    """
    Compute topographic derivatives from elevation.
    
    Args:
        surface_elev: elevation map (m)
        pixel_scale_m: cell size (m)
    
    Returns:
        Dict with slope, aspect, curvature fields
    """
    E = surface_elev
    
    # Gradient
    dEy, dEx = np.gradient(E, pixel_scale_m, pixel_scale_m)
    slope_mag = np.hypot(dEx, dEy) + 1e-12
    
    # Downslope aspect
    aspect = np.arctan2(-dEy, -dEx)
    
    # Laplacian (curvature)
    up = np.roll(E, -1, axis=0)
    down = np.roll(E, 1, axis=0)
    left = np.roll(E, 1, axis=1)
    right = np.roll(E, -1, axis=1)
    laplacian = (up + down + left + right - 4.0 * E) / (pixel_scale_m**2)
    
    return {
        'elevation': E,
        'dEx': dEx,
        'dEy': dEy,
        'slope': slope_mag,
        'aspect': aspect,
        'laplacian': laplacian,
    }

print("✓ BLOCK 1 loaded: Quantum RNG + Terrain Generation")
print("  Functions available:")
print("    - qrng_uint32() - Quantum random number generation")
print("    - quantum_seeded_topography() - Generate terrain")
print("    - compute_topo_fields() - Topographic analysis")

In [ ]:
"""
BLOCK 2: QUANTUM EROSION PHYSICS

This block contains:
- Quantum erosion decision system (3 modes: simple, entangled, amplitude)
- Flow routing (D8 algorithm)
- Stream power erosion law
- Sediment transport with capacity constraints
- Hillslope diffusion
- Complete simulation framework
"""

# ==============================================================================
# QUANTUM EROSION DECISION SYSTEM
# ==============================================================================

def create_quantum_erosion_mask(rain_field, threshold=0.1, batch_size=1000):
    """
    Create erosion decision mask using quantum Hadamard gates.
    
    MODE: SIMPLE (Independent decisions)
    
    For each cell where rain > threshold:
    - Create qubit in |0⟩
    - Apply Hadamard → (|0⟩ + |1⟩)/√2
    - Measure → 50% chance of 0 or 1
    - If 1: allow erosion at that cell
    
    Args:
        rain_field: 2D array of rainfall amounts
        threshold: minimum rain to trigger quantum decision
        batch_size: number of qubits to process at once
    
    Returns:
        erosion_mask: 2D boolean array where True = erosion happens
    """
    ny, nx = rain_field.shape
    erosion_mask = np.zeros((ny, nx), dtype=bool)
    
    active_cells = rain_field > threshold
    n_active = np.sum(active_cells)
    
    if n_active == 0:
        return erosion_mask
    
    if not HAVE_QISKIT:
        # Classical fallback: 50% probability
        erosion_mask[active_cells] = np.random.rand(n_active) > 0.5
        return erosion_mask
    
    # Quantum decision using Qiskit
    from qiskit import QuantumCircuit
    try:
        from qiskit_aer import Aer
    except ImportError:
        from qiskit import Aer
    
    active_indices = np.argwhere(active_cells)
    backend = Aer.get_backend('qasm_simulator')
    
    for start_idx in range(0, n_active, batch_size):
        end_idx = min(start_idx + batch_size, n_active)
        batch_n = end_idx - start_idx
        
        # Create circuit with batch_n qubits
        qc = QuantumCircuit(batch_n, batch_n)
        qc.h(range(batch_n))  # Hadamard on all qubits
        qc.measure(range(batch_n), range(batch_n))
        
        # Execute
        job = backend.run(qc, shots=1, memory=True)
        result = job.result()
        bitstring = result.get_memory(qc)[0][::-1]
        decisions = np.array([int(b) for b in bitstring], dtype=bool)
        
        # Apply to mask
        batch_indices = active_indices[start_idx:end_idx]
        for k, (i, j) in enumerate(batch_indices):
            erosion_mask[i, j] = decisions[k]
    
    return erosion_mask


def create_quantum_erosion_mask_entangled(rain_field, threshold=0.1, 
                                          entanglement_radius=2):
    """
    Enhanced quantum erosion with spatial entanglement.
    
    MODE: ENTANGLED (Correlated decisions)
    
    Neighboring cells are quantum-entangled, creating spatial correlation
    in erosion patterns (more realistic than independent decisions).
    
    Args:
        rain_field: 2D rainfall
        threshold: rain threshold
        entanglement_radius: how many neighbors to entangle
    
    Returns:
        erosion_mask: correlated erosion decisions
    """
    ny, nx = rain_field.shape
    erosion_mask = np.zeros((ny, nx), dtype=bool)
    active_cells = rain_field > threshold
    
    if not HAVE_QISKIT or np.sum(active_cells) == 0:
        return create_quantum_erosion_mask(rain_field, threshold)
    
    from qiskit import QuantumCircuit
    try:
        from qiskit_aer import Aer
    except ImportError:
        from qiskit import Aer
    
    backend = Aer.get_backend('qasm_simulator')
    processed = np.zeros((ny, nx), dtype=bool)
    
    # Process in small local neighborhoods
    for i in range(0, ny, entanglement_radius):
        for j in range(0, nx, entanglement_radius):
            i_end = min(i + entanglement_radius, ny)
            j_end = min(j + entanglement_radius, nx)
            
            local_active = active_cells[i:i_end, j:j_end]
            n_local = np.sum(local_active)
            
            if n_local == 0 or n_local > 10:
                continue
            
            # Create entangled circuit
            qc = QuantumCircuit(n_local, n_local)
            qc.h(range(n_local))  # Hadamard on all
            
            # Entangle with CNOT chain
            for q in range(n_local - 1):
                qc.cx(q, q+1)
            
            qc.measure(range(n_local), range(n_local))
            
            # Execute
            job = backend.run(qc, shots=1, memory=True)
            result = job.result()
            bitstring = result.get_memory(qc)[0][::-1]
            decisions = np.array([int(b) for b in bitstring], dtype=bool)
            
            # Map back to grid
            local_indices = np.argwhere(local_active)
            for k, (li, lj) in enumerate(local_indices):
                gi, gj = i + li, j + lj
                if not processed[gi, gj]:
                    erosion_mask[gi, gj] = decisions[k]
                    processed[gi, gj] = True
    
    # Fill remaining cells
    remaining = active_cells & ~processed
    if np.any(remaining):
        remaining_mask = create_quantum_erosion_mask(
            np.where(remaining, rain_field, 0), threshold
        )
        erosion_mask[remaining] = remaining_mask[remaining]
    
    return erosion_mask


def create_quantum_erosion_mask_amplitude(rain_field, threshold=0.1):
    """
    Quantum erosion with amplitude encoding.
    
    MODE: AMPLITUDE (Rain-intensity modulated)
    
    Rain intensity modulates the quantum amplitude, creating
    non-uniform erosion probabilities:
    - Higher rain → higher probability of erosion
    - Uses Ry rotation gates to encode rain intensity
    
    This is the most physically realistic quantum mode.
    
    Args:
        rain_field: 2D rainfall array
        threshold: minimum rain
    
    Returns:
        erosion_mask: amplitude-weighted erosion decisions
    """
    ny, nx = rain_field.shape
    erosion_mask = np.zeros((ny, nx), dtype=bool)
    active_cells = rain_field > threshold
    
    if not HAVE_QISKIT or np.sum(active_cells) == 0:
        return create_quantum_erosion_mask(rain_field, threshold)
    
    from qiskit import QuantumCircuit
    try:
        from qiskit_aer import Aer
    except ImportError:
        from qiskit import Aer
    
    backend = Aer.get_backend('qasm_simulator')
    active_indices = np.argwhere(active_cells)
    rain_max = rain_field.max()
    
    # Process with amplitude encoding
    for idx, (i, j) in enumerate(active_indices):
        rain_val = rain_field[i, j]
        rain_norm = min(rain_val / rain_max, 1.0)
        
        # Create single-qubit circuit
        qc = QuantumCircuit(1, 1)
        
        # Ry rotation: angle = π * rain_norm
        # rain_norm=0 → angle=0 → |0⟩ (no erosion)
        # rain_norm=1 → angle=π → |1⟩ (certain erosion)
        # rain_norm=0.5 → angle=π/2 → equal superposition (50% erosion)
        angle = np.pi * rain_norm
        qc.ry(angle, 0)
        qc.measure(0, 0)
        
        # Execute
        job = backend.run(qc, shots=1, memory=True)
        result = job.result()
        measurement = int(result.get_memory(qc)[0])
        
        erosion_mask[i, j] = (measurement == 1)
    
    return erosion_mask

# ==============================================================================
# FLOW ROUTING
# ==============================================================================

def compute_flow_direction_d8(elevation, pixel_scale_m):
    """Compute D8 flow direction (steepest descent)."""
    ny, nx = elevation.shape
    flow_dir = np.full((ny, nx), -1, dtype=np.int8)
    receivers = np.full((ny, nx, 2), -1, dtype=np.int32)
    
    di = np.array([-1, -1, 0, 1, 1, 1, 0, -1])
    dj = np.array([0, 1, 1, 1, 0, -1, -1, -1])
    distances = np.array([1, np.sqrt(2), 1, np.sqrt(2), 
                         1, np.sqrt(2), 1, np.sqrt(2)]) * pixel_scale_m
    
    for i in range(ny):
        for j in range(nx):
            z_center = elevation[i, j]
            steepest_slope = 0.0
            steepest_dir = -1
            
            for k in range(8):
                ni = i + di[k]
                nj = j + dj[k]
                if ni < 0 or ni >= ny or nj < 0 or nj >= nx:
                    continue
                
                dz = z_center - elevation[ni, nj]
                slope = dz / distances[k]
                
                if slope > steepest_slope:
                    steepest_slope = slope
                    steepest_dir = k
            
            if steepest_dir >= 0:
                flow_dir[i, j] = steepest_dir
                receivers[i, j, 0] = i + di[steepest_dir]
                receivers[i, j, 1] = j + dj[steepest_dir]
    
    return flow_dir, receivers


def compute_flow_accumulation(elevation, flow_dir, receivers, 
                              pixel_scale_m, rainfall=None):
    """Compute discharge (upslope contributing area × runoff)."""
    ny, nx = elevation.shape
    cell_area = pixel_scale_m ** 2
    
    if rainfall is not None:
        runoff = rainfall * 0.5  # 50% runoff
        water = runoff * cell_area
    else:
        water = np.ones((ny, nx)) * cell_area
    
    discharge = water.copy()
    
    # Topological sort
    indices = [(i, j) for i in range(ny) for j in range(nx)]
    indices_sorted = sorted(indices, key=lambda idx: elevation[idx], reverse=True)
    
    # Accumulate flow downhill
    for (i, j) in indices_sorted:
        if flow_dir[i, j] >= 0:
            ni, nj = receivers[i, j]
            if 0 <= ni < ny and 0 <= nj < nx:
                discharge[ni, nj] += discharge[i, j]
    
    return discharge


def route_flow(elevation, pixel_scale_m, rainfall=None):
    """Complete flow routing pipeline."""
    flow_dir, receivers = compute_flow_direction_d8(elevation, pixel_scale_m)
    discharge = compute_flow_accumulation(elevation, flow_dir, receivers, 
                                         pixel_scale_m, rainfall)
    dy, dx = np.gradient(elevation, pixel_scale_m)
    slope = np.sqrt(dx**2 + dy**2)
    slope = np.maximum(slope, 1e-6)
    
    return {
        'flow_dir': flow_dir,
        'receivers': receivers,
        'discharge': discharge,
        'slope': slope,
    }

# ==============================================================================
# EROSION PHYSICS
# ==============================================================================

def compute_stream_power_erosion(discharge, slope, K_base, m=0.5, n=1.0):
    """Stream power erosion: E = K * Q^m * S^n"""
    Q_norm = discharge / (discharge.max() + 1e-12)
    erosion = K_base * (Q_norm ** m) * (slope ** n)
    return erosion


def apply_hillslope_diffusion(elevation, pixel_scale_m, kappa, dt):
    """Hillslope diffusion: ∂h/∂t = κ ∇²h"""
    laplacian = (
        np.roll(elevation, -1, axis=0) +
        np.roll(elevation, 1, axis=0) +
        np.roll(elevation, -1, axis=1) +
        np.roll(elevation, 1, axis=1) -
        4 * elevation
    ) / (pixel_scale_m ** 2)
    
    delta_h = kappa * laplacian * dt
    return elevation + delta_h


def route_sediment(elevation, flow_dir, receivers, erosion_potential, 
                   erosion_mask, pixel_scale_m, transport_capacity_factor=1.2):
    """
    Route sediment downstream with capacity constraints.
    
    At each cell:
    - Compare sediment supply from upstream vs local transport capacity
    - Deposit if oversupplied
    - Erode if undersupplied (and quantum mask allows)
    """
    ny, nx = elevation.shape
    erosion_actual = np.zeros((ny, nx))
    sediment_supply = np.zeros((ny, nx))
    
    indices = [(i, j) for i in range(ny) for j in range(nx)]
    indices_sorted = sorted(indices, key=lambda idx: elevation[idx], reverse=True)
    
    for (i, j) in indices_sorted:
        capacity = erosion_potential[i, j] * transport_capacity_factor
        supply = sediment_supply[i, j]
        can_erode = erosion_mask[i, j]
        
        if supply > capacity:
            # Oversupply → deposit
            deposit = supply - capacity
            erosion_actual[i, j] = -deposit
            sediment_out = capacity
        elif can_erode:
            # Undersupply + erosion allowed → erode
            erode_amount = min(erosion_potential[i, j], capacity - supply)
            erosion_actual[i, j] = erode_amount
            sediment_out = supply + erode_amount
        else:
            # No erosion allowed by quantum decision
            erosion_actual[i, j] = 0.0
            sediment_out = supply
        
        # Route downstream
        if flow_dir[i, j] >= 0:
            ni, nj = receivers[i, j]
            if 0 <= ni < ny and 0 <= nj < nx:
                sediment_supply[ni, nj] += sediment_out
    
    return erosion_actual

# ==============================================================================
# QUANTUM EROSION SIMULATOR
# ==============================================================================

class QuantumErosionSimulator:
    """Complete quantum erosion simulation system."""
    
    def __init__(self, elevation, pixel_scale_m, 
                 K_base=5e-4, m=0.5, n=1.0, kappa=0.01):
        self.elevation = elevation.copy()
        self.initial_elevation = elevation.copy()
        self.pixel_scale_m = pixel_scale_m
        self.K_base = K_base
        self.m = m
        self.n = n
        self.kappa = kappa
        self.history = []
        
    def generate_rainfall(self, mean=1.0, std=0.3, seed=None):
        """Generate spatially variable rainfall field."""
        ny, nx = self.elevation.shape
        rng = np.random.default_rng(seed)
        rain = rng.normal(mean, std, size=(ny, nx))
        rain = ndimage.gaussian_filter(rain, sigma=5)
        rain = np.maximum(rain, 0.0)
        return rain
    
    def step(self, rainfall, dt=1.0, quantum_mode='amplitude', 
             rain_threshold=0.1, verbose=False):
        """Single erosion timestep."""
        if verbose:
            print(f"  Routing flow...")
        
        flow = route_flow(self.elevation, self.pixel_scale_m, rainfall)
        
        if verbose:
            print(f"  Computing erosion potential...")
        erosion_potential = compute_stream_power_erosion(
            flow['discharge'], flow['slope'], self.K_base, self.m, self.n
        )
        
        if verbose:
            print(f"  Creating quantum erosion mask ({quantum_mode})...")
        
        if quantum_mode == 'entangled':
            erosion_mask = create_quantum_erosion_mask_entangled(
                rainfall, rain_threshold
            )
        elif quantum_mode == 'amplitude':
            erosion_mask = create_quantum_erosion_mask_amplitude(
                rainfall, rain_threshold
            )
        else:
            erosion_mask = create_quantum_erosion_mask(
                rainfall, rain_threshold
            )
        
        if verbose:
            print(f"  Routing sediment...")
        erosion_actual = route_sediment(
            self.elevation, flow['flow_dir'], flow['receivers'],
            erosion_potential, erosion_mask, self.pixel_scale_m
        )
        
        self.elevation -= erosion_actual * dt
        
        if verbose:
            print(f"  Applying hillslope diffusion...")
        self.elevation = apply_hillslope_diffusion(
            self.elevation, self.pixel_scale_m, self.kappa, dt
        )
        
        # Statistics
        eroded = erosion_actual > 0
        deposited = erosion_actual < 0
        
        stats = {
            'erosion_actual': erosion_actual,
            'erosion_mask': erosion_mask,
            'discharge': flow['discharge'],
            'slope': flow['slope'],
            'total_erosion_m': np.sum(erosion_actual[eroded]),
            'total_deposition_m': -np.sum(erosion_actual[deposited]),
            'mean_erosion_m': np.mean(erosion_actual[eroded]) if np.any(eroded) else 0,
            'n_eroded_cells': np.sum(eroded),
            'n_deposited_cells': np.sum(deposited),
            'quantum_erosion_fraction': np.sum(erosion_mask) / erosion_mask.size,
        }
        
        self.history.append(stats)
        
        if verbose:
            print(f"  Erosion: {stats['total_erosion_m']:.3f} m total, "
                  f"{stats['n_eroded_cells']} cells")
            print(f"  Deposition: {stats['total_deposition_m']:.3f} m total, "
                  f"{stats['n_deposited_cells']} cells")
            print(f"  Quantum mask: {100*stats['quantum_erosion_fraction']:.1f}% active")
        
        return stats
    
    def run(self, n_steps=10, mean_rainfall=1.0, dt=1.0, 
            quantum_mode='amplitude', verbose=True):
        """Run multiple erosion steps."""
        if verbose:
            print(f"\nRunning quantum erosion simulation...")
            print(f"  Steps: {n_steps}")
            print(f"  Quantum mode: {quantum_mode}")
            print(f"  Mean rainfall: {mean_rainfall} m/year")
        
        for step_i in range(n_steps):
            if verbose:
                print(f"\nStep {step_i+1}/{n_steps}:")
            
            rainfall = self.generate_rainfall(mean=mean_rainfall, seed=step_i)
            self.step(rainfall, dt=dt, quantum_mode=quantum_mode, verbose=verbose)
        
        if verbose:
            total_change = np.sum(np.abs(self.elevation - self.initial_elevation))
            print(f"\n✓ Simulation complete!")
            print(f"  Total landscape change: {total_change:.2f} m")
    
    def get_erosion_map(self):
        """Get cumulative erosion map (positive = eroded, negative = deposited)."""
        return self.initial_elevation - self.elevation

print("✓ BLOCK 2 loaded: Quantum Erosion Physics")
print("  Functions available:")
print("    - create_quantum_erosion_mask() - Simple Hadamard")
print("    - create_quantum_erosion_mask_entangled() - With CNOT")
print("    - create_quantum_erosion_mask_amplitude() - Ry rotation")
print("    - QuantumErosionSimulator - Complete simulation")

In [ ]:
"""
BLOCK 3: DEMO + VISUALIZATION

This block contains:
- Visualization functions for all aspects of the simulation
- Complete demonstration workflow
- Comparison of different quantum modes
- Statistical analysis
"""

# ==============================================================================
# VISUALIZATION FUNCTIONS
# ==============================================================================

def plot_terrain_comparison(initial_elev, final_elev, pixel_scale_m, figsize=(18, 6)):
    """Plot before/after terrain comparison."""
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # Initial terrain
    im1 = axes[0].imshow(initial_elev, cmap='terrain', origin='lower')
    axes[0].set_title('Initial Terrain', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('X (cells)')
    axes[0].set_ylabel('Y (cells)')
    plt.colorbar(im1, ax=axes[0], label='Elevation (m)')
    
    # Final terrain
    im2 = axes[1].imshow(final_elev, cmap='terrain', origin='lower')
    axes[1].set_title('Final Terrain (After Erosion)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('X (cells)')
    axes[1].set_ylabel('Y (cells)')
    plt.colorbar(im2, ax=axes[1], label='Elevation (m)')
    
    # Erosion map
    erosion_map = initial_elev - final_elev
    vmax = np.percentile(np.abs(erosion_map), 98)
    im3 = axes[2].imshow(erosion_map, cmap='RdBu_r', origin='lower',
                        vmin=-vmax, vmax=vmax)
    axes[2].set_title('Cumulative Erosion/Deposition', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('X (cells)')
    axes[2].set_ylabel('Y (cells)')
    cbar = plt.colorbar(im3, ax=axes[2], label='Change (m)')
    cbar.ax.text(0.5, 0.05, 'Erosion', transform=cbar.ax.transAxes, 
                ha='center', fontsize=9, color='red')
    cbar.ax.text(0.5, 0.95, 'Deposition', transform=cbar.ax.transAxes, 
                ha='center', fontsize=9, color='blue')
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"\nTerrain Statistics:")
    print(f"  Initial elevation: {initial_elev.min():.1f} - {initial_elev.max():.1f} m")
    print(f"  Final elevation: {final_elev.min():.1f} - {final_elev.max():.1f} m")
    print(f"  Total erosion: {np.sum(erosion_map[erosion_map > 0]):.2f} m")
    print(f"  Total deposition: {-np.sum(erosion_map[erosion_map < 0]):.2f} m")
    print(f"  Max erosion: {erosion_map.max():.3f} m")
    print(f"  Max deposition: {-erosion_map.min():.3f} m")


def plot_flow_and_erosion(discharge, slope, erosion_map, figsize=(18, 6)):
    """Plot flow patterns and erosion."""
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # Discharge (log scale)
    discharge_log = np.log10(discharge + 1)
    im1 = axes[0].imshow(discharge_log, cmap='Blues', origin='lower')
    axes[0].set_title('Water Discharge (log scale)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('X (cells)')
    axes[0].set_ylabel('Y (cells)')
    plt.colorbar(im1, ax=axes[0], label='log₁₀(Q + 1)')
    
    # Slope
    im2 = axes[1].imshow(slope, cmap='hot', origin='lower')
    axes[1].set_title('Topographic Slope', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('X (cells)')
    axes[1].set_ylabel('Y (cells)')
    plt.colorbar(im2, ax=axes[1], label='Slope (m/m)')
    
    # Erosion
    vmax = np.percentile(np.abs(erosion_map), 98)
    im3 = axes[2].imshow(erosion_map, cmap='RdBu_r', origin='lower',
                        vmin=-vmax, vmax=vmax)
    axes[2].set_title('Erosion Pattern', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('X (cells)')
    axes[2].set_ylabel('Y (cells)')
    plt.colorbar(im3, ax=axes[2], label='Erosion (m)')
    
    plt.tight_layout()
    plt.show()


def plot_quantum_mask_effect(rainfall, erosion_mask, erosion_actual, figsize=(18, 6)):
    """Visualize quantum mask effect."""
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # Rainfall
    im1 = axes[0].imshow(rainfall, cmap='Blues', origin='lower')
    axes[0].set_title('Rainfall Field', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('X (cells)')
    axes[0].set_ylabel('Y (cells)')
    plt.colorbar(im1, ax=axes[0], label='Rain (m/year)')
    
    # Quantum mask
    im2 = axes[1].imshow(erosion_mask, cmap='RdYlGn_r', origin='lower')
    axes[1].set_title('Quantum Erosion Mask\n(Hadamard Decision)', 
                     fontsize=14, fontweight='bold')
    axes[1].set_xlabel('X (cells)')
    axes[1].set_ylabel('Y (cells)')
    cbar = plt.colorbar(im2, ax=axes[1], ticks=[0, 1])
    cbar.set_ticklabels(['No Erosion', 'Erosion'])
    
    # Actual erosion
    vmax = np.percentile(np.abs(erosion_actual), 98)
    im3 = axes[2].imshow(erosion_actual, cmap='RdBu_r', origin='lower',
                        vmin=-vmax, vmax=vmax)
    axes[2].set_title('Actual Erosion/Deposition', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('X (cells)')
    axes[2].set_ylabel('Y (cells)')
    plt.colorbar(im3, ax=axes[2], label='Change (m)')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nQuantum Mask Statistics:")
    print(f"  Cells with rain > threshold: {np.sum(rainfall > 0.1)}")
    print(f"  Cells allowed to erode (quantum): {np.sum(erosion_mask)}")
    if np.sum(rainfall > 0.1) > 0:
        print(f"  Erosion probability: {100*np.sum(erosion_mask)/np.sum(rainfall > 0.1):.1f}%")


def plot_3d_terrain(elevation, pixel_scale_m, title='Terrain', figsize=(12, 10),
                   azim=-60, elev=30):
    """3D visualization of terrain."""
    from mpl_toolkits.mplot3d import Axes3D
    
    ny, nx = elevation.shape
    x = np.arange(nx) * pixel_scale_m
    y = np.arange(ny) * pixel_scale_m
    X, Y = np.meshgrid(x, y)
    
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')
    
    # Subsample for performance
    stride = max(1, nx // 100)
    surf = ax.plot_surface(X[::stride, ::stride], Y[::stride, ::stride], 
                           elevation[::stride, ::stride],
                           cmap='terrain', linewidth=0, antialiased=True,
                           alpha=0.9)
    
    ax.set_xlabel('X (m)', fontsize=12)
    ax.set_ylabel('Y (m)', fontsize=12)
    ax.set_zlabel('Elevation (m)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.view_init(elev=elev, azim=azim)
    
    fig.colorbar(surf, ax=ax, shrink=0.5, label='Elevation (m)')
    plt.tight_layout()
    plt.show()

# ==============================================================================
# DEMONSTRATION
# ==============================================================================

print("✓ BLOCK 3 loaded: Visualization + Demo")
print("\n" + "=" * 80)
print("QUANTUM EROSION SIMULATION - DEMO")
print("=" * 80)

# Parameters
N = 128  # Grid size (use 256 or 512 for higher resolution)
pixel_scale_m = 10.0  # 10m per cell
elev_range_m = 500.0  # 500m elevation range
seed = 42

print(f"\n1. GENERATING TERRAIN")
print(f"   Grid size: {N} × {N}")
print(f"   Domain: {N*pixel_scale_m/1000:.2f} km × {N*pixel_scale_m/1000:.2f} km")
print(f"   Cell size: {pixel_scale_m} m")

# Generate normalized terrain
z_norm, rng = quantum_seeded_topography(
    N=N, beta=3.2, warp_amp=0.10, ridged_alpha=0.15, random_seed=seed
)

# Scale to actual elevation
initial_elevation = z_norm * elev_range_m

print(f"   ✓ Terrain generated!")
print(f"   Elevation range: {initial_elevation.min():.1f} - {initial_elevation.max():.1f} m")

# Visualize initial terrain
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(initial_elevation, cmap='terrain', origin='lower')
ax.set_title('Initial Quantum-Seeded Terrain', fontsize=16, fontweight='bold')
ax.set_xlabel('X (cells)', fontsize=12)
ax.set_ylabel('Y (cells)', fontsize=12)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Elevation (m)', fontsize=12)
plt.tight_layout()
plt.show()

print(f"\n2. RUNNING QUANTUM EROSION SIMULATION")
print("   Creating simulator...")

# Create simulator
sim = QuantumErosionSimulator(
    elevation=initial_elevation,
    pixel_scale_m=pixel_scale_m,
    K_base=5e-4,  # Erodibility coefficient
    m=0.5,        # Discharge exponent
    n=1.0,        # Slope exponent
    kappa=0.01    # Hillslope diffusion
)

# Run simulation
sim.run(
    n_steps=5,              # Number of erosion events
    mean_rainfall=1.0,      # Mean rainfall (m/year)
    dt=1.0,                 # Timestep (years)
    quantum_mode='amplitude',  # Options: 'simple', 'entangled', 'amplitude'
    verbose=True
)

print("\n3. VISUALIZING RESULTS")

# Get final state
final_elevation = sim.elevation
erosion_map = sim.get_erosion_map()

# Plot 1: Before/After comparison
print("\n   a) Terrain Comparison (Before/After)")
plot_terrain_comparison(initial_elevation, final_elevation, pixel_scale_m)

# Plot 2: Flow and erosion patterns
if len(sim.history) > 0:
    print("\n   b) Flow and Erosion Patterns (Last Step)")
    last_step = sim.history[-1]
    plot_flow_and_erosion(
        last_step['discharge'], 
        last_step['slope'], 
        last_step['erosion_actual']
    )

# Plot 3: Quantum mask effect
if len(sim.history) > 0:
    print("\n   c) Quantum Mask Effect (Last Step)")
    last_step = sim.history[-1]
    rainfall = sim.generate_rainfall(mean=1.0, seed=len(sim.history)-1)
    plot_quantum_mask_effect(
        rainfall,
        last_step['erosion_mask'],
        last_step['erosion_actual']
    )

# Statistical Summary
print("\n4. STATISTICAL SUMMARY")
print(f"\n   Overall Landscape Change:")
print(f"     Total volume eroded: {np.sum(erosion_map[erosion_map > 0]) * pixel_scale_m**2:.2e} m³")
print(f"     Total volume deposited: {-np.sum(erosion_map[erosion_map < 0]) * pixel_scale_m**2:.2e} m³")
print(f"     Net change: {np.sum(erosion_map) * pixel_scale_m**2:.2e} m³")

print(f"\n   Elevation Statistics:")
print(f"     Initial mean elevation: {initial_elevation.mean():.2f} m")
print(f"     Final mean elevation: {final_elevation.mean():.2f} m")
print(f"     Mean elevation change: {final_elevation.mean() - initial_elevation.mean():.3f} m")
print(f"     Initial relief: {initial_elevation.max() - initial_elevation.min():.2f} m")
print(f"     Final relief: {final_elevation.max() - final_elevation.min():.2f} m")

print(f"\n   Per-Step Summary:")
for i, step in enumerate(sim.history):
    print(f"     Step {i+1}: Erosion={step['total_erosion_m']:.4f}m, "
          f"Deposition={step['total_deposition_m']:.4f}m, "
          f"Quantum={100*step['quantum_erosion_fraction']:.1f}%")

print("\n" + "=" * 80)
print("✓ SIMULATION COMPLETE!")
print("=" * 80)
print("\nTo compare different quantum modes, run:")
print("  for mode in ['simple', 'entangled', 'amplitude']:")
print("      sim = QuantumErosionSimulator(initial_elevation, pixel_scale_m)")
print("      sim.run(n_steps=3, quantum_mode=mode)")
print("      plot_terrain_comparison(initial_elevation, sim.elevation, pixel_scale_m)")